In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import re

In [2]:
# We test on alabama, before we automate the proces
web_url = 'https://www.rentdata.org/states/district-of-columbia/2015'
rent_res = requests.get(web_url)

In [3]:
soup_rent = BeautifulSoup(rent_res.content)
rent_table = soup_rent.find_all('table', class_ = 'hover scroll tablesorter')
rent_table

[<table class="hover scroll tablesorter" id="state_table">
 <thead>
 <tr>
 <th style="width: 250px">County</th>
 <th><span aria-haspopup="true" class="has-tip top" data-disable-hover="false" data-tooltip="" tabindex="7" title="Studio, or efficiency.">0 BR</span></th>
 <th>1 BR</th>
 <th>2 BR</th>
 <th>3 BR</th>
 <th><span aria-haspopup="true" class="has-tip top" data-disable-hover="false" data-tooltip="" tabindex="9" title="Home or apartment with 4 bedrooms. Add 15% for each additional bedroom.">4 BR</span></th>
 <th><span aria-haspopup="true" class="has-tip top" data-disable-hover="false" data-tooltip="" tabindex="10" title="Census Bureau Population Estimates">Est. Population</span></th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td><a href="https://www.rentdata.org/washington-arlington-alexandria-dc-va-md-hud-metro-fmr-area/2015" title="Average rent prices in District of Columbia"><b>District of Columbia</b></a>
 <abbr title="Metropolitan Area"><span class="label info float-right left-spaced"

In [4]:
# We now want to find all the gattdang counties i tell you hwat 
county_table = soup_rent.find_all('b')
county_table
county_list = []
for item in county_table:
    county_list.append(item.text.strip())
state_name = county_list[0]
county_list = county_list[3:]
state_name

'District of Columbia'

In [7]:
# collect rents for the different types of buildings 
regex = re.compile('[%s]' % re.escape("[-,\.!?:’]"))
cost_table = soup_rent.find_all('span', class_ = 'money')
cost_table = cost_table[5:]
cost_table
cost_list1 = []
cost_list2 = []
state_list = []
# convert it into str first
for item in cost_table:
    print(item)
    cost_list1.append(item.text.split()[0])
    state_list.append(state_name)
# remove $, then turn into float 
for item in cost_list1:
    money_text = regex.sub("", item.split('$')[1])
    money_float = float(money_text)
    cost_list2.append(money_float)
print(cost_list1)
cost_list2

<span class="money">$1,230</span>
<span class="money">$1,458</span>
<span class="money">$1,951</span>
<span class="money">$2,451</span>
['$1,230', '$1,458', '$1,951', '$2,451']


[1230.0, 1458.0, 1951.0, 2451.0]

In [ ]:
# make separate BR lists 
ind0 = 0
ind1 = 1
ind2 = 2
ind3 = 3
ind4 = 4

br0 = []
br1 = []
br2 = []
br3 = []
br4 = []
while ind0 in range(len(cost_list2)):
    br0.append(cost_list2[ind0])
    br1.append(cost_list2[ind1])
    br2.append(cost_list2[ind2])
    br3.append(cost_list2[ind3])
    br4.append(cost_list2[ind4])
    ind0 += 5
    ind1 += 5
    ind2 += 5
    ind3 += 5
    ind4 += 5
ala_rent_df = pd.DataFrame(list(zip(state_list, county_list, br0, br1, br2, br3, br4)), columns = ['State Name', 'County Name', 'Rent for 0 Bedrooms',
                                                                                     'Rent for 1 Bedroom', 'Rent for 2 Bedrooms',
                                                                                     'Rent for 3 Bedrooms', 'Rent for 4 Bedrooms'])
ala_rent_df.head()